# Timer Contract

In [1]:
%%writefile src/Timer.sol

pragma solidity ^0.8.0;
contract Timer {
    uint256 startTime;
    function start() public {
        //startTime=now; now is deprecated as of v0.7
        startTime=block.timestamp;
    }
    function timePassed() public view returns(uint256) {
        //return now-startTime;
        return block.timestamp-startTime;
    }
    function getNow() view public returns(uint) {
        //return now;
        return block.timestamp;
    }
}

Writing src/Timer.sol


In [4]:
!solc --optimize --combined-json abi,bin src/Timer.sol > src/Timer.json

--> src/Timer.sol



In [5]:
!cat src/Timer.json

{"contracts":{"src/Timer.sol:Timer":{"abi":[{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}],"bin":"608060405234801561001057600080fd5b5060d38061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605e575b600080fd5b60476068565b60405190815260200160405180910390f35b426047565b606642600055565b005b60008054607490426079565b905090565b600082821015609857634e487b7160e01b600052601160045260246000fd5b50039056fea26469706673582212203c2b59ce533913d7519507fd7b51bc8d99f58b2e1451b04d4450e6c66fab485764736f6c634300080c0033"}},"version":"0.8.12+commit.f00d7308.Darwin.appleclang"}


In [6]:
%%writefile src/TimerImportTest.js
const _abiBinJson = require('./Timer.json');     //importing a javascript file
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Timer.sol:Timer']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: ", _abi);
console.log("- ABIArray: ", _abiArray);
console.log("- Bytecode: ", _bin);

Writing src/TimerImportTest.js


In [7]:
!node src/TimerImportTest.js

- contract name:  [ 'src/Timer.sol:Timer' ]
- ABI:  [
  {
    inputs: [],
    name: 'getNow',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  },
  {
    inputs: [],
    name: 'start',
    outputs: [],
    stateMutability: 'nonpayable',
    type: 'function'
  },
  {
    inputs: [],
    name: 'timePassed',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  }
]
- ABIArray:  [
  {
    inputs: [],
    name: 'getNow',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  },
  {
    inputs: [],
    name: 'start',
    outputs: [],
    stateMutability: 'nonpayable',
    type: 'function'
  },
  {
    inputs: [],
    name: 'timePassed',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  }
]
- Bytecode:  608060405234801561001057600080fd5b5060d38061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605e575b60

In [18]:
%%writefile src/TimerDeployAbiBinFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Timer.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Timer.sol:Timer']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi)); // -> 문자열로 바꾸고 parsing해야함.
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! 
_bin=_abiBinJson.contracts[contractName].bin;

console.log("- ABI: " + _abiArray);
console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/TimerDeployAbiBinFromFile.js


In [19]:
!node src/TimerDeployAbiBinFromFile.js

- contract name:  [ 'src/Timer.sol:Timer' ]
- ABI: [object Object],[object Object],[object Object]
- Bytecode: 608060405234801561001057600080fd5b5060d38061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605e575b600080fd5b60476068565b60405190815260200160405180910390f35b426047565b606642600055565b005b60008054607490426079565b905090565b600082821015609857634e487b7160e01b600052601160045260246000fd5b50039056fea26469706673582212203c2b59ce533913d7519507fd7b51bc8d99f58b2e1451b04d4450e6c66fab485764736f6c634300080c0033
Deploying the contract from 0x6F973e6C3758Bd4445dB01ed92A33Fe6f148e7A5
hash: 0x8e7f5c2533ba175e5ec5c7eb54a5c1ecda9c3594124ae9b3b1c05eae59e5e130
---> The contract deployed to: 0xCB3149700d0B761cB5c99376e4EAFc4607e87f0c


In [22]:
%%writefile src/timerUse.js
var Web3=require('web3');
var _abiBinJson = require('./Timer.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Timer.sol:Timer']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

var timer = new web3.eth.Contract(_abiArray,"0xCB3149700d0B761cB5c99376e4EAFc4607e87f0c");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Call from: " + accounts[0]);
    timer.methods.getNow().call().then(function(value) {console.log("Now: ", value);});
    await timer.methods.start().send({from:accounts[0],gas:100000});
    //await 4000;
    timer.methods.timePassed().call().then(function(value) {console.log("Passed: ", value);});
}

doIt()

Overwriting src/timerUse.js


In [25]:
!node src/timerUse.js

Call from: 0x6F973e6C3758Bd4445dB01ed92A33Fe6f148e7A5
Now:  1650269676
Passed:  0
